<a href="https://colab.research.google.com/github/JADukeAIPI/AIPIProject/blob/main/dbpopulate_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3 as db
import numpy as np
#Import .py modules with processed data: readcsv contains Kaggle data, WebScrapingAtlantaEvents contains Discover Atlanta event data,
#and APICALL contains Budget Car Rental data
import readcsv
import WebScrapingAtlantaEvents
import APICALL
import LuxuryCarAPIMultipleDates

In [ ]:
# Creates a blank db if one does not exist, otherwise opens connection to rentaldb, and drops the tables Kaggle, API, and Webscrape
conn = db.connect('rentaldb')
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS Kaggle")
c.execute("DROP TABLE IF EXISTS API")
c.execute("DROP TABLE IF EXISTS LuxuryAPI")
c.execute("DROP TABLE IF EXISTS AllCarsAPI")
c.execute("DROP TABLE IF EXISTS Webscrape")
c.execute("DROP TABLE IF EXISTS LargeEvents")

In [ ]:
#Runs the sort_lux_df function from the readcsv import. This function reads the CarRentalDataV1 csv data into a dataframe
#sorts it based on location (Atlanta), then filters the vehciles to only show luxury brands/models.
csvdf = readcsv.sort_lux_df()
#This injects the sort_lux_df into the Kaggle table of the rentaldb 
csvdf.to_sql('Kaggle', conn, if_exists='replace')

#Runs the get_events function from the WebScrapingAtlantaEvents import. This function scrapes Discover Atlanta, 
#and formats the events
webdf = WebScrapingAtlantaEvents.get_events('2022-11-18', '2023-03-03')
#This injects the get_events data into the Webscrape table of the rentaldb 
webdf.to_sql('Webscrape', conn, if_exists='replace')

#Gets large venue events at the Mercedes-Benz stadium and State Farm Arena, and injects into the LargeEvents table of the rentaldb
large_venue_query = 'Location.str.contains("State Farm Arena|Mercedes-Benz Stadium")'
web_large_venue_df = WebScrapingAtlantaEvents.get_events('2022-11-18', '2023-03-03', query_string=large_venue_query)
web_large_venue_df.to_sql('LargeEvents', conn, if_exists='replace')

#Runs the api_request function from APICALL. This function calls the API, and reads the data as JSON, then turns it into a df
df2 = APICALL.api_request()
#Runs the clean_api_df function from APICALL. This function cleans the df2 dataframe that the api_request function created
apidf = APICALL.clean_api_df(df2)
#This injects the cleaned API data into API table of the rentaldb
apidf.to_sql('API', conn, if_exists='replace')

#Runs the get_bookings_multiple_dates function from LuxuryCarAPIMultipleDates. This function calls the API and requests the specified days of booking information.
#We will retrieve all cars and inject into AllCarsAPI, and luxury cars and inject into LuxuryAPI
all_cars_apidf = LuxuryCarAPIMultipleDates.get_bookings_multiple_dates('2022-11-18', '2023-03-03')
all_cars_apidf.to_sql('AllCarsAPI', conn, if_exists='replace')
all_cars_apidf.to_csv('AllCarsAPI.csv')
all_cars_apidf.to_pickle('AllCarsAPI.pkl')

luxuryapidf = all_cars_apidf.loc[all_cars_apidf['vehicle_info.group']=='Luxury'] 
luxuryapidf.to_sql('LuxuryAPI', conn, if_exists='replace')
luxuryapidf.to_csv('LuxuryAPI.csv')
luxuryapidf.to_pickle('LuxuryAPI.pkl')


658

In [ ]:
#Verifying the tables are in the rentaldb
csvdf = pd.read_sql_query("SELECT * FROM Kaggle", conn)
apidf = pd.read_sql_query("SELECT * FROM API", conn)
luxuryapidf = pd.read_sql_query("SELECT * FROM LuxuryAPI", conn)
scrapedf = pd.read_sql_query("SELECT * FROM Webscrape", conn)
all_cars_apidf = pd.read_sql_query("SELECT * FROM AllCarsAPI", conn)
web_large_venue_df = pd.read_sql_query("SELECT * FROM LargeEvents", conn)

print('Kaggle Data')
display(csvdf.head())
print('API Data')
display(apidf.head())
print('All Car DAta')
display(all_cars_apidf.head())
print('Luxury API Data')
display(luxuryapidf.head())
print('Webscrape Data')
display(scrapedf.head())
print('Large Venue Events')
display(web_large_venue_df)

Kaggle Data


,index,fuelType,rating,renterTripsTaken,reviewCount,city,country,latitude,location.longitude,location.state,owner.id,daily_rate,make,model,type,year,airportcity
0,41,GASOLINE,5.00,12,10,Grayson,US,33.906905,-83.960234,GA,1295660,175,Mercedes-Benz,S-Class,car,2015,Atlanta
1,42,GASOLINE,5.00,2,2,Atlanta,US,33.891295,-84.453929,GA,12796797,200,BMW,7 Series,car,2018,Atlanta
2,50,GASOLINE,4.96,152,140,Atlanta,US,33.927684,-84.343055,GA,511103,155,Chevrolet,Corvette Stingray,car,2014,Atlanta
3,54,GASOLINE,5.00,23,21,Atlanta,US,33.740676,-84.358236,GA,8752492,209,Audi,RS 7,car,2014,Atlanta
4,57,ELECTRIC,4.88,18,16,Duluth,US,33.970512,-84.161462,GA,8609349,159,Tesla,Model S,car,2018,Atlanta


API Data


,index,Vehicle_id,Vehicle_Name,Category,Transmission,Dropoff_City,Base_Price,Pickup_City
0,270,644881573,Cadillac XTS,Luxury,Automatic,,374.66,
1,209,673928006,Mercedes-Benz GLC,Luxury,Automatic,,367.62,
2,263,636479888,Cadillac XTS,Luxury,Automatic,,361.38,
3,214,646119413,Cadillac XTS,Luxury,Automatic,,350.17,
4,149,674291426,Lincoln Continental,Luxury,Automatic,,348.82,


Luxury API Data


,index,Vehicle_id,Vehicle Name,Category,Transmission,Base Price,Price,Date Pickup,Date Dropoff
0,208,644881573,Cadillac XTS,Luxury,Automatic,171.26,178.20,2022-11-18,2022-11-19
1,237,636479888,Cadillac XTS,Luxury,Automatic,164.51,171.18,2022-11-18,2022-11-19
2,215,646119413,Cadillac XTS,Luxury,Automatic,158.72,165.16,2022-11-18,2022-11-19
3,232,644901168,Cadillac XTS,Luxury,Automatic,153.78,160.02,2022-11-18,2022-11-19
4,235,636492688,Cadillac XTS,Luxury,Automatic,147.14,153.11,2022-11-18,2022-11-19


Webscrape Data


,index,Event,Dates,Location,Description
0,1773,Atlanta Brewery Tour,2022-11-19 00:00:00,Iron Hill Brewery – Buckhead,Tour by Southern Culinary Tours! Join us as we...
1,1774,Free Birds–$ 2.00 Fall Flick at Aurora Cineplex,2022-11-19 00:00:00,Aurora Cineplex,FREE BIRDS ( PG)–our $ 2.00 Fall Family Flicks...
2,1775,Retreat into Oneness,2022-11-19 00:00:00,Elohee Retreat Center,Retreat into Oneness offers a respite from the...
3,1776,Winter Farmers Market,2022-11-19 00:00:00,Market Pavilion,We have so much more than produce. Fill your m...
4,1777,“In Hope for a Better Future” Art Exhibition,2022-11-19 00:00:00,Hammonds House Museum,Hammonds House Museum closes out 2022 with a d...
